In [10]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions, KeywordsOptions
# from .. import credentials_path
import sys
sys.path.append("..")
import config

In [11]:
def get_ibm_nlu():
    """ Constructs an instance of NaturalLanguageUnderstandingV1 using the tokens in the file
    at credentials_path
    Returns:
        An instance of NaturalLanguageUnderstandingV1
    """
    authenticator = IAMAuthenticator(config.watson_api_key)
    nlu = NaturalLanguageUnderstandingV1(
        version='2019-07-12',
        authenticator=authenticator
    )
    nlu.set_service_url(config.watson_service_url)
    return nlu

In [34]:
nlu_api = get_ibm_nlu()

In [13]:
def sentiment_analysis(input_text, nlu):
    """
    Args:
    input_text .... The plain text of a tweet for a candidate
    nlu ... The natural language understanding tool from IBM Watson
    Returns:
    A python dictionary mapping the tweet text to all of the sentiment scores
    """
    tweet_score = nlu.analyze(text=input_text, features=Features(
        emotion=EmotionOptions(),
        sentiment=SentimentOptions(),
        keywords=KeywordsOptions()),
        language='en').get_result()
    return tweet_score

In [14]:
sentiment_analysis("I need help. Please heelp. I'm scared. #911", nlu_api)

{'emotion': {'document': {'emotion': {'anger': 0.026052,
    'disgust': 0.00719,
    'fear': 0.719166,
    'joy': 0.014699,
    'sadness': 0.115116}}},
 'keywords': [{'count': 1, 'relevance': 0.775061, 'text': 'help'}],
 'language': 'en',
 'sentiment': {'document': {'label': 'negative', 'score': -0.984533}},
 'usage': {'features': 3, 'text_characters': 43, 'text_units': 1}}

In [5]:
def sentiment_analysis_list(tweets_list, nlu):
    """
    Args:
      tweets_list .... The plain text list of tweets for a candidate
      nlu ... The natural language understanding tool from IBM Watson
    Returns:
      A python dictionary mapping the tweet text to all of the sentiment scores
    """
    senti_scores = {}
    for i, tweet in enumerate(tweets_list):
        try:
            tweet_scores = nlu.analyze(text=tweets_list[i], features=Features(
                emotion=EmotionOptions(),
                sentiment=SentimentOptions(),
                keywords=KeywordsOptions()), language='en').get_result()
            senti_scores[tweet] = tweet_scores
        except:
            senti_scores[tweet] = 'N/A, could not parse'
    return senti_scores

In [44]:
# sentence = "I just saw a guy run over a man. Scary! #Police #CPD" # This is a good example
sentence = "I am so afraid. The cops just shot a guy. He is bleeding"
anlys = sentiment_analysis(sentence, nlu_api)
anlys

{'emotion': {'document': {'emotion': {'anger': 0.072451,
    'disgust': 0.272918,
    'fear': 0.852308,
    'joy': 0.002377,
    'sadness': 0.059568}}},
 'keywords': [{'count': 1, 'relevance': 0.903788, 'text': 'cops'},
  {'count': 1, 'relevance': 0.700506, 'text': 'guy'},
  {'count': 1, 'relevance': 0.386169, 'text': 'bleeding'}],
 'language': 'en',
 'sentiment': {'document': {'label': 'negative', 'score': -0.973419}},
 'usage': {'features': 3, 'text_characters': 56, 'text_units': 1}}

In [31]:
anlys['emotion']['document']['emotion']

{'anger': 0.091166,
 'disgust': 0.339515,
 'fear': 0.78847,
 'joy': 0.020769,
 'sadness': 0.064783}

In [36]:
def is_fearful(input_text, nlu):
    tweet_score = nlu.analyze(text=input_text, 
                              features=Features(emotion=EmotionOptions()),language='en').get_result()
    return True if tweet_score["emotion"]["document"]["emotion"]["fear"]>0.4 else False

In [41]:
is_fearful(sentence, nlu_api)

True

In [40]:
# Putting them as functions
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions
import sys
sys.path.append("..")
import config


def get_ibm_nlu():
    """ Constructs an instance of NaturalLanguageUnderstandingV1 using the tokens in the file
    at credentials_path
    Returns:
        An instance of NaturalLanguageUnderstandingV1
    """
    authenticator = IAMAuthenticator(config.watson_api_key)
    nlu = NaturalLanguageUnderstandingV1(
        version='2019-07-12',
        authenticator=authenticator
    )
    nlu.set_service_url(config.watson_service_url)
    return nlu

def is_fearful(input_text, nlu):
    """ Classifies the text based on the fear level. Above a threshold is fearful. 
    Returns:
        True or False if fear detected
    """
    tweet_score = nlu.analyze(text=input_text, 
                              features=Features(emotion=EmotionOptions()),language='en').get_result()
    return True if tweet_score["emotion"]["document"]["emotion"]["fear"]>0.4 else False

In [ ]:
"I am so afraid. The cops just shot a guy. He is bleeding"